In [ ]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools import FunctionTool
from google.adk.apps.app import App, ResumabilityConfig
from google.genai import types
import numpy as np
import pandas as pd
import asyncio
import os
import sys
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")


In [41]:
!ls /kaggle/working/src/agents

adk_orchestrator.py  __init__.py      __pycache__
detector_agent.py    memory_agent.py


In [42]:
# API key setup
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key
print("✓ API key loaded from Kaggle secrets")

✓ API key loaded from Kaggle secrets


In [43]:
#Creating project structure

!mkdir -p /kaggle/working/src/agents
!mkdir -p /kaggle/working/src/tools
!mkdir -p /kaggle/working/src/utils
!mkdir -p /kaggle/working/src/data/rag

Path("/kaggle/working/src/__init__.py").touch()
Path("/kaggle/working/src/agents/__init__.py").touch()
Path("/kaggle/working/src/tools/__init__.py").touch()
Path("/kaggle/working/src/utils/__init__.py").touch()

print("✓ Project structure created\n")


✓ Project structure created



In [44]:
# RAG File
breathing_content = """Box Breathing Exercise (4-4-4-4)

This technique is used by Navy SEALs to stay calm under pressure.

Steps:
1. Sit comfortably with your back straight and feet flat on the floor
2. Breathe in slowly through your nose for 4 counts
3. Hold your breath for 4 counts
4. Breathe out slowly through your mouth for 6 counts
5. Hold empty for 4 counts
6. Repeat this cycle 5-10 times

Why it works: Slow breathing activates your parasympathetic nervous system, signaling your body to relax.

4-7-8 Breathing (The Relaxing Breath)

Steps:
1. Exhale completely through your mouth
2. Inhale through your nose for 4 counts
3. Hold for 7 counts
4. Exhale through mouth for 8 counts
5. Repeat 4 cycles
"""

grounding_content = """5-4-3-2-1 Grounding Technique

This brings you back to the present moment when anxiety disconnects you.

Steps:
Acknowledge FIVE things you can SEE
Acknowledge FOUR things you can TOUCH
Acknowledge THREE things you can HEAR
Acknowledge TWO things you can SMELL
Acknowledge ONE thing you can TASTE

Why it works: Redirects attention from anxious thoughts to immediate sensory experiences.

Cold Water Method:
Splash cold water on your face or hold an ice cube. The shock activates the mammalian dive reflex, instantly calming your nervous system.

Progressive Muscle Relaxation:
Tense and release muscle groups:
- Clench fists (5 seconds), release
- Tighten arms, release
- Tense shoulders, release
"""

reassurance_content = """Calming Truths for Anxiety

When Feeling Panic:
- "This is temporary. Panic attacks peak within 10 minutes."
- "I am not dying. This is anxiety, not danger."
- "My body is protecting me, though there's no real threat."
- "I've survived 100% of my panic attacks."

When Overwhelmed:
- "I only need to focus on this moment."
- "I can do hard things. I've done them before."
- "It's okay to not be okay right now."
- "My feelings are valid."

Cognitive Reframing:
Instead of: "I can't handle this"
Try: "This is hard, but I'm handling it right now"

Instead of: "What if everything goes wrong?"
Try: "What if things go okay?"

The 90-Second Rule:
Strong emotions trigger chemicals that flush out within 90 seconds. If you're still feeling it after that, you're re-triggering it with your thoughts.
"""

with open("/kaggle/working/src/data/rag/breathing.txt", "w") as f:
    f.write(breathing_content)
with open("/kaggle/working/src/data/rag/grounding.txt", "w") as f:
    f.write(grounding_content)
with open("/kaggle/working/src/data/rag/reassurance.txt", "w") as f:
    f.write(reassurance_content)

print("✓ RAG content created (3 files)\n")

✓ RAG content created (3 files)



In [45]:
#Creating core agent classes

detector_code = '''import re
import json
from ..utils.llm_wrapper import llm_call

FALLBACK_KEYWORDS = {
    "panic": ["panic", "can't breathe", "chest racing", "heart racing"],
    "anxious": ["anxious", "anxiety", "worried", "nervous"],
    "stressed": ["stressed", "overwhelmed"],
    "sad": ["sad", "depressed", "down"],
    "positive": ["good", "great", "okay", "well"]
}

class DetectorAgent:
    def __init__(self, use_llm=True, model_kwargs=None):
        self.use_llm = use_llm
        self.model_kwargs = model_kwargs or {}

    def _fallback_classify(self, text):
        text_l = text.lower()
        emotion = "neutral"
        for e, keywords in FALLBACK_KEYWORDS.items():
            for kw in keywords:
                if kw in text_l:
                    emotion = e
                    break
            if emotion != "neutral":
                break

        intensity = {"panic": 5, "anxious": 3, "stressed": 3, "sad": 2, "positive": 1}.get(emotion, 0)
        needs_intervention = intensity >= 4 or emotion == "panic"
        
        return {
            "emotion": emotion,
            "intensity": intensity,
            "trigger_tags": [],
            "needs_intervention": needs_intervention
        }

    def classify(self, text):
        if self.use_llm:
            prompt = f"""Classify emotion from text. Return ONLY JSON:
{{"emotion": "panic|anxious|stressed|sad|neutral|positive", "intensity": 0-5, "trigger_tags": [], "needs_intervention": true|false}}

Text: "{text}"

JSON:"""
            try:
                out = llm_call(prompt, **(self.model_kwargs))
                out = out.replace("```json", "").replace("```", "").strip()
                return json.loads(out)
            except:
                return self._fallback_classify(text)
        return self._fallback_classify(text)
'''

memory_code = '''import json
import os
from datetime import datetime
from ..utils.timestamp import timestamp

class MemoryAgent:
    def __init__(self, path="/kaggle/working/src/data/sample_logs.json"):
        self.path = path
        if not os.path.exists(self.path):
            with open(self.path, "w") as f:
                json.dump([], f)

    def _read(self):
        with open(self.path, "r") as f:
            try:
                return json.load(f)
            except:
                return []

    def _write(self, data):
        with open(self.path, "w") as f:
            json.dump(data, f, indent=2)

    def save_entry(self, user_id, text, detector_out, intervention=None, notes=None, session_id=None):
        data = self._read()
        entry = {
            "id": timestamp(),
            "user_id": user_id,
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "text": text,
            "emotion": detector_out.get("emotion"),
            "intensity": detector_out.get("intensity"),
            "trigger_tags": detector_out.get("trigger_tags"),
            "intervention": intervention,
            "notes": notes or ""
        }
        data.append(entry)
        self._write(data)
        return entry

    def load_last_n(self, n=7):
        return self._read()[-n:]

    def compact_summary(self, last_k=30):
        data = self._read()[-last_k:]
        if not data:
            return "No memory yet."
        emotions = {}
        total = 0
        for d in data:
            emotions[d.get("emotion", "neutral")] = emotions.get(d.get("emotion", "neutral"), 0) + 1
            total += d.get("intensity", 0)
        avg = total / max(1, len(data))
        top = ", ".join([f"{k}({v})" for k, v in sorted(emotions.items(), key=lambda x: x[1], reverse=True)[:3]])
        return f"Last {len(data)} entries avg intensity {avg:.2f}. Common: {top}."
'''

with open("/kaggle/working/src/agents/detector_agent.py", "w") as f:
    f.write(detector_code)
with open("/kaggle/working/src/agents/memory_agent.py", "w") as f:
    f.write(memory_code)

print("✓ Core agent classes created\n")

✓ Core agent classes created



In [46]:
# Creating Utility files
timestamp_code = '''from datetime import datetime

def timestamp():
    return datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
'''

llm_wrapper_code = '''import os
import google.generativeai as genai

def llm_call(prompt, model="gemini-2.5-flash", max_output_tokens=512, temperature=0.0, **kwargs):
    """Call Gemini API"""
    if not os.environ.get("GOOGLE_API_KEY"):
        raise RuntimeError("GOOGLE_API_KEY not set")
    
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
    model_client = genai.GenerativeModel(model)
    
    response = model_client.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            max_output_tokens=max_output_tokens,
            temperature=temperature
        )
    )
    
    return response.text
'''

with open("/kaggle/working/src/utils/timestamp.py", "w") as f:
    f.write(timestamp_code)
with open("/kaggle/working/src/utils/llm_wrapper.py", "w") as f:
    f.write(llm_wrapper_code)

print("✓ Utility files created\n")

✓ Utility files created



In [47]:
#Creating RAG tool

rag_code = '''import os
import glob

class RAG:
    def __init__(self, path="/kaggle/working/src/data/rag"):
        self.path = path
        self.docs = []
        self._load()

    def _load(self):
        self.docs = []
        if not os.path.exists(self.path):
            return
        for fn in glob.glob(os.path.join(self.path, "*.txt")):
            with open(fn, "r", encoding="utf-8") as f:
                self.docs.append({"file": os.path.basename(fn), "text": f.read().strip()})

    def doc_count(self):
        return len(self.docs)

    def retrieve(self, query, top_k=1):
        q_tokens = set(query.lower().split())
        scored = []
        for d in self.docs:
            score = len(q_tokens & set(d["text"].lower().split()))
            scored.append((score, d["text"]))
        scored.sort(key=lambda x: x[0], reverse=True)
        results = [s[1] for s in scored if s[0] > 0]
        return results[:top_k] if results else ([self.docs[0]["text"]] if self.docs else [""])
'''

with open("/kaggle/working/src/tools/rag.py", "w") as f:
    f.write(rag_code)

print("✓ RAG tool created\n")

✓ RAG tool created



In [ ]:
# Creating ADK Orchestrator Agent

import sys
if 'src.agents.adk_orchestrator' in sys.modules:
    del sys.modules['src.agents.adk_orchestrator']

orchestrator_code = """import os
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.function_tool import FunctionTool
from google.adk.apps.app import App, ResumabilityConfig

# TOOLS

async def detect_emotion(text: str) -> dict:
    '''Analyzes emotional state and returns emotion, intensity, needs_intervention'''
    from ..agents.detector_agent import DetectorAgent
    detector = DetectorAgent(use_llm=True)
    result = detector.classify(text)
    print(f"  Detected: {result.get('emotion')} (intensity {result.get('intensity')})")
    return result


async def retrieve_technique(intensity: int) -> str:
    '''Retrieves calming technique based on intensity level'''
    from ..tools.rag import RAG
    rag = RAG()
    
    if intensity >= 4:
        results = rag.retrieve("panic breathing exercise", top_k=1)
        print(f"  Retrieved panic breathing technique")
    elif intensity >= 2:
        results = rag.retrieve("anxiety grounding technique", top_k=1)
        print(f"  Retrieved grounding technique")
    else:
        return "No specific technique needed."
    
    return results[0] if results else "Technique not found."


async def save_to_memory(user_id: str, text: str, emotion: str, intensity: int) -> dict:
    '''Saves mood entry to memory for pattern tracking'''
    from ..agents.memory_agent import MemoryAgent
    memory = MemoryAgent()
    
    detector_out = {
        "emotion": emotion,
        "intensity": intensity,
        "trigger_tags": []
    }
    
    entry = memory.save_entry(user_id, text, detector_out)
    print(f"  Saved to memory")
    return {"status": "saved", "entry_id": entry['id']}


async def get_mood_history(last_n: int = 7) -> str:
    '''Gets recent mood history and patterns'''
    from ..agents.memory_agent import MemoryAgent
    memory = MemoryAgent()
    
    entries = memory.load_last_n(last_n)
    if not entries:
        return "No previous mood logs."
    
    summary = "Recent mood logs:\\\\n"
    for e in entries[-3:]:
        summary += f"- {e.get('timestamp', 'N/A')[:10]}: {e.get('emotion')} (intensity {e.get('intensity')})\\\\n"
    
    return summary + "\\\\n" + memory.compact_summary(last_n)

# AGENT CREATION

retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)


def create_orchestrator(user_id="demo_user"):
    '''Create Anxiety Buddy with ADK'''
    
    print(f"✓ Initialized Anxiety Buddy for user: {user_id}")
    
    # Create LlmAgent
    anxiety_agent = LlmAgent(
        name="anxiety_buddy",
        model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
        instruction=f'''You are Anxiety Support Buddy, a warm and compassionate AI companion for user {user_id}.

WORKFLOW FOR EVERY MESSAGE:

1. Call detect_emotion(user's message) to get emotion and intensity
2. Based on intensity:
   - If 4-5: Call retrieve_technique(intensity) for panic breathing
   - If 2-3: Call retrieve_technique(intensity) for grounding
3. Call save_to_memory(user_id="{user_id}", text=message, emotion=detected, intensity=detected)
4. Write a supportive response:

FOR PANIC (intensity 4-5):
"I'm here with you. You're safe. Let's try this breathing exercise:
[Share the technique in numbered steps]
If you're in danger, call 911 or text 988.
Try this now. Let me know how you feel."

FOR ANXIETY (intensity 2-3):
"I hear you - that sounds difficult. Let's try this technique:
[Share the technique]
Would you like to try this, or talk more?"

FOR MILD (intensity 0-1):
"[Acknowledge warmly]
I'm here to listen. Would you like to talk or try a calming technique?"

SPECIAL:
- "How am I doing?" → Call get_mood_history()

STYLE: Warm, clear, 3-6 sentences. Remind: "I'm support, not therapy."
CRITICAL: Always provide a text response after calling tools.''',
        tools=[
            FunctionTool(func=detect_emotion),
            FunctionTool(func=retrieve_technique),
            FunctionTool(func=save_to_memory),
            FunctionTool(func=get_mood_history)
        ]
    )
    
    # Wrap in App
    agent = App(
        name="anxiety_buddy_app",
        root_agent=anxiety_agent,
        resumability_config=ResumabilityConfig(is_resumable=True)
    )
    
    return agent
"""

with open("/kaggle/working/src/agents/adk_orchestrator.py", "w") as f:
    f.write(orchestrator_code)

print("✓ ADK orchestrator created with YOUR original logic")
print("✓ Only fixed: ADK syntax (async, FunctionTool, App wrapper)")
print("✓ Tools: detect_emotion, retrieve_technique, save_to_memory, get_mood_history")

✓ ADK orchestrator created with YOUR original logic
✓ Only fixed: ADK syntax (async, FunctionTool, App wrapper)
✓ Tools: detect_emotion, retrieve_technique, save_to_memory, get_mood_history


In [ ]:
# Testing

sys.path.insert(0, "/kaggle/working")

# Clear cache
for mod in list(sys.modules.keys()):
    if mod.startswith('src.'):
        del sys.modules[mod]

from src.agents.adk_orchestrator import create_orchestrator
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import uuid

print("Creating Anxiety Buddy App...")
anxiety_app = create_orchestrator(user_id="kaggle_demo")

print("Creating session service and runner...")
session_service = InMemorySessionService()
runner = Runner(app=anxiety_app, session_service=session_service)

print("✓ System initialized\n")

# HELPER FUNCTION


def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"🤖 Agent > {part.text}")
                # Debug: show tool execution
                if part.function_response and "status" in str(part.function_response.response):
                    print(f"   [Tool Output]: {part.function_response.response}")

# WORKFLOW FUNCTION

async def run_anxiety_workflow(user_query: str, auto_confirm: bool = True):
    """Runs the agent, handles pauses, simulates confirmation, and resumes."""
    
    print(f"\n{'='*60}")
    print(f"🗣️ User > {user_query}\n")

    # Create unique session
    session_id = f"session_{uuid.uuid4().hex[:8]}"
    await session_service.create_session(
        app_name="anxiety_buddy_app", 
        user_id="test_user", 
        session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=user_query)])
    events = []

    # PASS 1: Initial Request
    async for event in runner.run_async(
        user_id="test_user", 
        session_id=session_id, 
        new_message=query_content
    ):
        events.append(event)

    # CHECK FOR PAUSE 
    approval_request = None
    invocation_id = None
    
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.function_call and part.function_call.name == "adk_request_confirmation":
                    approval_request = part.function_call
                    invocation_id = event.invocation_id
                    break
    
    # PASS 2: Handle Approval
    if approval_request:
        hint_text = approval_request.args.get('hint', 'Confirmation Required')
        print(f"⏸️  SYSTEM PAUSED: {hint_text}")
        print(f"🤔 Human Decision: {'✅ CONFIRM' if auto_confirm else '❌ REJECT'}\\n")
        
        # Create confirmation response
        confirmation_response = types.FunctionResponse(
            id=approval_request.id,
            name="adk_request_confirmation",
            response={"confirmed": auto_confirm}
        )
        resume_message = types.Content(
            role="user", 
            parts=[types.Part(function_response=confirmation_response)]
        )

        # RESUME the agent
        async for event in runner.run_async(
            user_id="test_user", 
            session_id=session_id, 
            new_message=resume_message, 
            invocation_id=invocation_id
        ):
            events.append(event)

    # FINAL OUTPUT
    print_agent_response(events)
    print(f"{'='*60}\\n")

print("✓ Workflow engine ready\n")


Creating Anxiety Buddy App...
✓ Initialized Anxiety Buddy for user: kaggle_demo
Creating session service and runner...
✓ System initialized

✓ Workflow engine ready



In [55]:
print("--- 🤖 STARTING INTEGRATION TESTS ---")

# TEST 1: Emergency Protocol
print("\\n🔹 TEST 1: Emergency Protocol")
await run_anxiety_workflow("Help! I'm having a panic attack and can't breathe!")
await asyncio.sleep(5)

# TEST 2: Anxiety Support
print("\\n🔹 TEST 2: Anxiety Support (with technique)")
await run_anxiety_workflow("I'm really anxious about my exam tomorrow.")
await asyncio.sleep(5)

# TEST 3: Mood Logging (Stateful)
print("\\n🔹 TEST 3: Mood Logging (with confirmation)")
await run_anxiety_workflow("I want to log that I'm feeling stressed, intensity 3/5", auto_confirm=True)
await asyncio.sleep(5)

# TEST 4: Pattern Inquiry
print("\\n🔹 TEST 4: Mood History Retrieval")
await run_anxiety_workflow("How have my moods been?")
await asyncio.sleep(5)



--- 🤖 STARTING INTEGRATION TESTS ---
\n🔹 TEST 1: Emergency Protocol

🗣️ User > Help! I'm having a panic attack and can't breathe!

  Detected: panic (intensity 5)
  Retrieved panic breathing technique
  Saved to memory
   [Tool Output]: {'status': 'saved', 'entry_id': '20251130T161757Z'}
🤖 Agent > I'm here with you. You're safe. Let's try this breathing exercise:

**Box Breathing Exercise (4-4-4-4)**
1. Sit comfortably with your back straight and feet flat on the floor.
2. Breathe in slowly through your nose for 4 counts.
3. Hold your breath for 4 counts.
4. Breathe out slowly through your mouth for 6 counts.
5. Hold empty for 4 counts.
6. Repeat this cycle 5-10 times.

If you're in danger, call 911 or text 988. Try this now. Let me know how you feel. Remember, I'm here for support, not as a replacement for professional therapy.
============================================================\n
\n🔹 TEST 2: Anxiety Support (with technique)

🗣️ User > I'm really anxious about my exam tomorro

In [57]:
print("\n🔹 TEST 5: Mild (Intensity 0-1)")
await run_anxiety_workflow("I had a little bit of trouble sleeping last night.")
await asyncio.sleep(10) 

# TEST 6: Moderate Anxiety (Intensity 2-3)
print("\n🔹 TEST 6: Moderate Anxiety (Intensity 2-3)")
await run_anxiety_workflow("I have a big deadline next week and I'm really starting to feel the pressure.")
await asyncio.sleep(5) 

# TEST 7: History Retrieval (Check Memory Interaction)
print("\n🔹 TEST 7: Mood History Retrieval")
await run_anxiety_workflow("Show me my recent emotional patterns.")

print("\\n✅ ALL TESTS COMPLETE!")


🔹 TEST 5: Mild (Intensity 0-1)

🗣️ User > I had a little bit of trouble sleeping last night.

  Detected: neutral (intensity 0)
  Saved to memory
   [Tool Output]: {'status': 'saved', 'entry_id': '20251130T162021Z'}
🤖 Agent > It sounds like you're feeling a bit restless. I'm here to listen if you'd like to talk more about what's on your mind or perhaps try a calming technique that might help you relax. Remember, I'm here for support, not as a replacement for therapy.
============================================================\n

🔹 TEST 6: Moderate Anxiety (Intensity 2-3)

🗣️ User > I have a big deadline next week and I'm really starting to feel the pressure.

  Detected: stressed (intensity 3)
  Retrieved grounding technique
  Saved to memory
   [Tool Output]: {'status': 'saved', 'entry_id': '20251130T162039Z'}
🤖 Agent > I hear you - that sounds difficult. It's completely understandable to feel pressure with a big deadline approaching.

Let's try a grounding technique to help you fee